## 수치 미분을 이용한 심층 신경망 학습

In [1]:
import time
import numpy as np

## 유틸리티 함수

In [2]:
def _t(x):
    return np.transpose(x)

def _m(A, B):
    return np.matmul(A, B)

## Sigmoid 구현

In [3]:
class Sigmoid:
    def __init__(self):
        self.last_o = 1. # 초기값을 1로 하는 이유는 다른 값과 곱해야하기 때문이다. 0으로 하면 다른 값을 곱했을 때 0이 되기 때문.

    def __call__(self, x):  # 순전파 연산
        self.last_o = 1 / (1.0 + np.exp(-x))
        return self.last_o

    def grad(self):  # sigmoid(x)(1-sigmoid(x))  # 역전파 미분 연산
        return self.last_o * (1 - self.last_o)

## Mean Squared Error 구현

In [4]:
class MeanSquaredError:
    def __init__(self):
        # gradient
        self.dh = 1
        self.last_diff = 1
        
    def __call__(self, h, y):  # 1/2 * mean((h - y)^2)
        self.last_diff = h - y
        return 1 / 2 * np.mean(np.square(h - y))

    def grad(self):  # h - y
        return self.last_diff

## 뉴런 구현

In [5]:
class Neuron:
    def __init__(self, W, b, a_obj):
        # Model parameters
        self.W = W
        self.b = b
        self.a = a_obj()
        
        # gradient
        self.dW = np.zeros_like(self.W)
        self.db = np.zeros_like(self.b)
        self.dh = np.zeros_like(_t(self.W))
        
        self.last_x = np.zeros((self.W.shape[0]))
        self.last_h = np.zeros((self.W.shape[1]))
        
    def __call__(self, x):
        self.last_x = x
        self.last_h = _m(_t(self.W), x) + self.b
        return self.a(self.last_h)

    def grad(self):  # y = Wx + b ==> dy/dh = W  # layer의 출력과 입력 미분
        return self.W * self.a.grad()

    def grad_W(self, dh):  # loss_function을 W로 미분 한 것을 저장
        grad = np.ones_like(self.W)
        grad_a = self.a.grad()
        for j in range(grad.shape[1]):  # dy/dW = x    # [1] : output dimension
            grad[:, j] = dh[j] * grad_a[j] * self.last_x
            # dh : 누적된 gradient 중 마지막 gradient
            # grad_a : 현제 activation의 gradient
            # self.last_x : w로 미분한 gradient : 마지막으로 입력받은 gradient
        return grad
        
    def grad_b(self, dh):  # 지금까지의 미분을 다 곱한것을 입력으로 받아서 loss_function을 b로 미분
        return dh * self.a.grad()  # dy/db = 1  # dh * self.a.grad() * 1

## 심층신경망 구현

In [6]:
class DNN:
    def __init__(self, hidden_depth, num_neuron, input, output, activation=Sigmoid):
        def init_var(i, o):
            return np.random.normal(0.0, 0.01, (i, o)), np.zeros((o,))

        self.sequence = list()
        # First hidden layer
        W, b = init_var(input, num_neuron)
        self.sequence.append(Neuron(W, b, activation))

        # Hidden Layers
        for index in range(hidden_depth):
            W, b = init_var(num_neuron, num_neuron)
            self.sequence.append(Neuron(W, b, activation))

        # Output Layer
        W, b = init_var(num_neuron, output)
        self.sequence.append(Neuron(W, b, activation))

    def __call__(self, x):
        for layer in self.sequence:
            x = layer(x)
        return x

    def calc_gradient(self, loss_obj):  # 마지막 loss_function에 대한 gradient를 미리 계산해서 저장
        loss_obj.dh = loss_obj.grad()
        self.sequence.append(loss_obj)  # for문으로 loss_obj를 처리하기 위해 시퀀스에 넣어준다.
        
        # back-prop loop
        for i in range(len(self.sequence) - 1, 0, -1):  # gradient를 반대로 하나씩 계산
            l1 = self.sequence[i]
            l0 = self.sequence[i - 1]
            
            l0.dh = _m(l0.grad(), l1.dh)  # l0.dh : loss를 현재 layer로 미분한 값. # l0.grad() : 이 다음 layer를 현재 layer로 미분하는 gradient
            l0.dW = l0.grad_W(l1.dh)
            l0.db = l0.grad_b(l1.dh)
        
        self.sequence.remove(loss_obj)  # 다 처리 후 다시 빼줘야 한다. 그렇지 않으면 나중에 이 함수를 call할때 losss_obj가 항상 들어있기 때문에 출력을 얻지 못하고 loss만 얻기 때문.

## 경사하강 학습법

In [7]:
def gradient_descent(network, x, y, loss_obj, alpha=0.01):
    loss = loss_obj(network(x), y)  # Forward inference
    network.calc_gradient(loss_obj)  # Back-propagation
    for layer in network.sequence:
        layer.W += -alpha * layer.dW
        layer.b += -alpha * layer.db
    return loss

## 동작 테스트

In [8]:
x = np.random.normal(0.0, 1.0, (10,))
y = np.random.normal(0.0, 1.0, (2,))

t = time.time()
dnn = DNN(hidden_depth=5, num_neuron=32, input=10, output=2, activation=Sigmoid)
loss_obj = MeanSquaredError()
for epoch in range(100):
    loss = gradient_descent(dnn, x, y, loss_obj, alpha=0.01)
    print('Epoch {}: Test loss {}'.format(epoch, loss))
print('{} seconds elapsed.'.format(time.time() - t))

Epoch 0: Test loss 0.5797287879748814
Epoch 1: Test loss 0.5732411988680092
Epoch 2: Test loss 0.5668387705002067
Epoch 3: Test loss 0.5605247717273485
Epoch 4: Test loss 0.5543021396435517
Epoch 5: Test loss 0.5481734804383686
Epoch 6: Test loss 0.542141072922351
Epoch 7: Test loss 0.5362068744865803
Epoch 8: Test loss 0.5303725292407148
Epoch 9: Test loss 0.5246393780607514
Epoch 10: Test loss 0.519008470271563
Epoch 11: Test loss 0.5134805766896638
Epoch 12: Test loss 0.5080562037577893
Epoch 13: Test loss 0.5027356085138507
Epoch 14: Test loss 0.4975188141517355
Epoch 15: Test loss 0.49240562594934323
Epoch 16: Test loss 0.48739564735933516
Epoch 17: Test loss 0.482488296079488
Epoch 18: Test loss 0.4776828199416092
Epoch 19: Test loss 0.47297831248003175
Epoch 20: Test loss 0.46837372806227084
Epoch 21: Test loss 0.463867896485062
Epoch 22: Test loss 0.4594595369583895
Epoch 23: Test loss 0.4551472714180391
Epoch 24: Test loss 0.4509296371234953
Epoch 25: Test loss 0.4468050985126